In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Test with default hyperparameters

In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = xgb.XGBClassifier()
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('08-xgboost','XGBClassifier','1-Default', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('08-xgboost','XGBClassifier','1-Default', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)

# result 24/01/2025
#train f1 score: 0.1902 mcc score: 0.3061 roc auc score: 0.5533
#test  f1 score: 0.0972 mcc score: 0.1638 roc auc score: 0.5269

# Scaling choice
with xgboost the scaling has no influence ?

In [ ]:
%%script false
# scaling choice

import matplotlib.pyplot as plt
import numpy as np

predictors = fd.getPredictors(dfTrx)
parameters={}
#parameters['max_iter']=1000

modelClf = xgb.XGBClassifier()
duration_in_s,f1Train,f1Test, scaler = fd.processModel(modelClf,dfTrx, predictors,[], parameters, scaler=None)
scalers = fd.getScalers()
for key in scalers:
    print(key)
    scaler=scalers.get(key)
    modelClf = xgb.XGBClassifier()
    duration_in_s,f1Train,f1Test, scaler = fd.processModel(modelClf,dfTrx, predictors, [],parameters,scaler=scaler)

# scaler has no adding value for the result

# Hyperparameters tuning
starting point
n_estimators=50,learning_rate=1

In [ ]:
%%script false
from scipy.stats import randint

modelClf = xgb.XGBClassifier()
# starting point n_estimators=100,  max_depth=7 
dic_param={
    'n_estimators':randint(250,420),
    'max_depth':randint(6,14), 
    'learning_rate':[0.1,0.2,0.3],
    'subsample':[0.8,1.0],
    'colsample_bytree':[0.8,1.0]
}

res=fd.hyperparameterSelectionRandomizedSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)
# result (24/02/2025)
#{'colsample_bytree': 1.0, 'learning_rate': 0.3, 'max_depth': 11, 'n_estimators': 412, 'subsample': 1.0}
#0.16193636516994936
#score   0.7304565353345841
#scoref1 0.7304565353345841

In [ ]:
%%script false
modelClf = xgb.XGBClassifier()

dic_param={
    'n_estimators':[290,310,330,350,370,390],
    'max_depth':[11,12,13,14,15],
    'learning_rate':[0.05,0.1,0.2,0.3],
    'subsample':[0.8,1.0],
    'colsample_bytree':[0.8,1.0]
}


res=fd.hyperparameterSelectionGridSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)

#24/02/2025
#{'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 11, 'n_estimators': 430, 'subsample': 0.8}
#0.16830577955695836
#scoref1 0.7415592387968079


In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
parameters= {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 11, 'n_estimators': 330, 'subsample': 0.8}


modelClf = xgb.XGBClassifier()
modelClf.set_params(**parameters)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('08-xgboost','XGBClassifier','2-After tuning', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('08-xgboost','XGBClassifier','2-After tuning', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)

## result  (26/01/2025)
#test f1 score: 0.1286 mcc score: 0.1592 roc auc score: 0.5403


In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result('08-xgboost','XGBClassifier','2-After tuning', f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])
